<b>#100DaysofMlCode<br>
<b>#Day 4<br>
<b>#Data Hackathon<br>
    <b>#Data Preprocess

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt # Visualization
import seaborn as sns # For easier statistical plotting
sns.set_style("whitegrid")

In [11]:
train = pd.read_csv('Yes_Bank_Training.csv')

In [ ]:
train_x = train[['serial_number','age_in_years','job_description','marital_status','education_details','has_default','balance_in_account','housing_status','previous_loan','phone_type','date','month_of_year','call_duration','campaign_contacts','days_passed','previous_contact','poutcome_of_campaign']]

In [ ]:
train_y = train['outcome']

In [ ]:
print(train_x.info())

In [ ]:
print(train_x.describe())

In [ ]:
pd.set_option('precision',2)
plt.figure(figsize=(15,12))
sns.heatmap(train_x.drop(["serial_number"],axis=1).corr(), square=True)
plt.suptitle("Pearson Correlation Heatmap")
plt.show()

In [ ]:
skews = train_x.skew()
for col in train_x.drop(["serial_number"], axis=1).columns.tolist():
    if (train_x[col].dtype == 'float64') or (train_x[col].dtype == 'int64') :
        sns.distplot(train_x[col].dropna(), kde=False, label="skew = " + str(round(skews[col],2)) )
        plt.legend()
        plt.show()
        
del skews        

In [ ]:
train.head(1)

In [15]:
def process_label(text):
    if text == 'yes':
        return 1
    else:
        return 0

In [ ]:
train['outcome'] = train['outcome'].apply(lambda x : process_label(x))

In [ ]:
train['outcome'].value_counts()

In [ ]:
count_no_sub = len(train[train['outcome']==0])
count_sub = len(train[train['outcome']==1])
# print(count_no_sub,count_sub)
pct_of_no_sub = count_no_sub/(count_no_sub+count_sub)
print("percentage of no subscription is", pct_of_no_sub*100)
pct_of_sub = count_sub/(count_no_sub+count_sub)
print("percentage of subscription", pct_of_sub*100)

In [ ]:
train.groupby('outcome').mean()

In [ ]:
train.groupby('job_description').mean()

In [ ]:
%matplotlib inline
pd.crosstab(train.job_description,train.outcome).plot(kind='bar')
plt.title('Purchase Frequency for Job Title')
plt.xlabel('Job')
plt.ylabel('Frequency of Purchase')
plt.savefig('purchase_fre_job')

In [ ]:
table=pd.crosstab(train.marital_status,train.outcome)
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title('Stacked Bar Chart of Marital Status vs Purchase')
plt.xlabel('Marital Status')
plt.ylabel('Proportion of Customers')
plt.savefig('mariral_vs_pur_stack')

In [ ]:
table=pd.crosstab(train.education_details,train.outcome)
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title('Stacked Bar Chart of Education vs Purchase')
plt.xlabel('Education')
plt.ylabel('Proportion of Customers')
plt.savefig('edu_vs_pur_stack')

In [ ]:
pd.crosstab(train.month_of_year,train.outcome).plot(kind='bar')
plt.title('Purchase Frequency for Day of Week')
plt.xlabel('Day of Week')
plt.ylabel('Frequency of Purchase')
plt.savefig('pur_dayofweek_bar')

In [ ]:
train.age_in_years.hist()
plt.title('Histogram of Age')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.savefig('hist_age')

In [ ]:
pd.crosstab(train.poutcome_of_campaign,train.outcome).plot(kind='bar')
plt.title('Purchase Frequency for Poutcome')
plt.xlabel('Poutcome')
plt.ylabel('Frequency of Purchase')
plt.savefig('pur_fre_pout_bar')

In [ ]:
print(train.info())

In [12]:
data = train
cat_vars = ['job_description','marital_status','education_details','has_default','housing_status','previous_loan','phone_type','month_of_year','poutcome_of_campaign']
for var in cat_vars:
    cat_list = 'var' + '_' + var
    cat_list = pd.get_dummies(train[var], prefix=var)
    data1 = pd.concat([data,cat_list], axis=1)
    data = data1

cat_vars = ['job_description','marital_status','education_details','has_default','housing_status','previous_loan','phone_type','month_of_year','poutcome_of_campaign']
data_vars = data.columns.values.tolist()
to_keep = [i for i in data_vars if i not in cat_vars]

In [13]:
Train = data[to_keep]

In [ ]:
Train.head()

In [16]:
Train['outcome'] = Train['outcome'].apply(lambda x : process_label(x))

D:\Programs\Anaconda2\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
Train.head()

In [ ]:
# Train.to_csv('training_set.csv',encoding='utf-8')

In [ ]:
# !pip install imblearn  

In [17]:
from imblearn.over_sampling import SMOTE

In [18]:
X = Train.loc[:, Train.columns != 'outcome']
Y = Train.loc[:, Train.columns == 'outcome']

os = SMOTE(random_state=0)
# X_train, X_val, Y_train, Y_val = train_test_split (X,Y,test_size=0.1,random_state=0)
columns = X.columns

In [19]:
os_data_X, os_data_Y = os.fit_sample(X,Y)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns)
os_data_Y = pd.DataFrame(data=os_data_Y,columns=['outcome'])

D:\Programs\Anaconda2\envs\tensorflow-gpu\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
len(os_data_Y)

In [ ]:
len(os_data_Y[os_data_Y['outcome']==0])

In [ ]:
os_data_X.info()

In [ ]:
os_data_Y.info()

In [20]:
data_final_vars = Train.columns.values.tolist()
y = ['outcome']
X = [i for i in data_final_vars if i not in y]

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

rfe = RFE(logreg,20)
rfe = rfe.fit(os_data_X, os_data_Y.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

D:\Programs\Anaconda2\envs\tensorflow-gpu\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Programs\Anaconda2\envs\tensorflow-gpu\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Programs\Anaconda2\envs\tensorflow-gpu\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Programs\Anaconda2\envs\tensorflow-gpu\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Programs\Anaconda2\envs\tensorflow-gpu\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarn

[False False False False False False False False  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True False False False  True False False False False False
 False False False False False False False False False False False False
 False False False]
[31 26 32 27 29 25 28 30  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1 24 23  2  1  4  3  8  7  6 22 18 20 21 13 17 16  9 15 19  5 10
 11 14 12]


In [21]:
select_index = np.where(rfe.ranking_ == 1)[0]

In [22]:
select_cols = os_data_X.columns[select_index]

In [23]:
select_cols

Index(['job_description_admin.', 'job_description_blue-collar',
       'job_description_entrepreneur', 'job_description_housemaid',
       'job_description_management', 'job_description_retired',
       'job_description_self-employed', 'job_description_services',
       'job_description_student', 'job_description_technician',
       'job_description_unemployed', 'job_description_unknown',
       'marital_status_divorced', 'marital_status_married',
       'marital_status_single', 'education_details_primary',
       'education_details_secondary', 'education_details_tertiary',
       'education_details_unknown', 'housing_status_yes'],
      dtype='object')

In [ ]:
train_x = os_data_X[select_cols]

In [ ]:
train_x

In [ ]:
train_y = os_data_Y

In [ ]:
train_y

In [ ]:
train_x.to_csv('train_x_rfe.csv')

In [ ]:
train_y.to_csv('train_y_rfe.csv')

In [ ]:
X = train_x
y = train_y

In [ ]:
# !pip install statsmodels
!pip install patsy

In [ ]:
import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

In [ ]:
y_pred = logreg.predict(X_val)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_val, y_val)))

<b><i>Preprocess Test Data

In [2]:
test = pd.read_csv('Yes_Bank_Test.csv',encoding='utf-8')

In [4]:
test.shape

(13562, 17)

In [6]:
data = test
cat_vars = ['job_description','marital_status','education_details','has_default','housing_status','previous_loan','phone_type','month_of_year','poutcome_of_campaign']
for var in cat_vars:
    cat_list = 'var' + '_' + var
    cat_list = pd.get_dummies(data[var], prefix=var)
    data1 = pd.concat([data,cat_list], axis=1)
    data = data1

cat_vars = ['job_description','marital_status','education_details','has_default','housing_status','previous_loan','phone_type','month_of_year','poutcome_of_campaign']
data_vars = data.columns.values.tolist()
to_keep = [i for i in data_vars if i not in cat_vars]

In [7]:
Test = data[to_keep]

In [10]:
Test.columns

Index(['serial_number', 'age_in_years', 'balance_in_account', 'date',
       'call_duration', 'campaign_contacts', 'days_passed', 'previous_contact',
       'job_description_admin.', 'job_description_blue-collar',
       'job_description_entrepreneur', 'job_description_housemaid',
       'job_description_management', 'job_description_retired',
       'job_description_self-employed', 'job_description_services',
       'job_description_student', 'job_description_technician',
       'job_description_unemployed', 'job_description_unknown',
       'marital_status_divorced', 'marital_status_married',
       'marital_status_single', 'education_details_primary',
       'education_details_secondary', 'education_details_tertiary',
       'education_details_unknown', 'has_default_no', 'has_default_yes',
       'housing_status_no', 'housing_status_yes', 'previous_loan_no',
       'previous_loan_yes', 'phone_type_cellular', 'phone_type_telephone',
       'phone_type_unknown', 'month_of_year_apr', '

In [24]:
test_x = Test[select_cols]

In [25]:
test_x.head()

,job_description_admin.,job_description_blue-collar,job_description_entrepreneur,job_description_housemaid,job_description_management,job_description_retired,job_description_self-employed,job_description_services,job_description_student,job_description_technician,job_description_unemployed,job_description_unknown,marital_status_divorced,marital_status_married,marital_status_single,education_details_primary,education_details_secondary,education_details_tertiary,education_details_unknown,housing_status_yes
0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1
1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1
3,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1
4,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1
